In [1]:
import qval as qv

In [2]:
A = qv.quantize(0.499999, 3)
A

qval(q='001', n=3, d='1', i='1')

In [3]:
qv.dequantize(A)

0.3333333333333333

In [4]:
a = qv.qval('010')
b = qv.qval('110')

a.__mul__(b)

qval(q='10100', n=5, d='20', i='-4')

In [5]:
q = qv.qval('001')

qv.qSigmoid(q)

qval(q='010', n=3, d='2', i='2')

In [6]:
n1 = qv.quantize(0.5, 8)
n2 = qv.quantize(0.5, 4)


print(qv.dequantize(qv.qAdd(n1, n2)))
print(qv.dequantize(qv.qMul(n1, n2)))

1.0
0.28517365561862906


In [7]:
import torch

a = torch.tensor(0.5)
b = torch.tensor(0.5)

qa = qv.quantize(a.item(), 8)
qb = qv.quantize(b.item(), 8)


print(qv.dequantize(qv.qAdd(qa, qb)))
print(qv.dequantize(qv.qMul(qa, qb)))

1.0
0.2500152597204419


In [8]:
in_len = 10
out_len = 1
layer = torch.nn.Linear(in_len, out_len)
input_test = torch.tensor([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0])

print(layer(input_test).item())

-0.1343405395746231


In [9]:
# Quantize layer
from qval import qval

in_params, out_params = layer.parameters()
in_params = list(in_params)

if in_len > 1:
    in_params = in_params[0]

q_in_params = []
for param in in_params:
    # q_in_params.append(qv.quantize(param.item(), 8))
    q_in_params.append(qval(param.item(), 8))

if out_len > 1:
    out_params = out_params[0]

q_out_params = []
for param in out_params:
    # q_out_params.append(qv.quantize(param.item(), 8))
    q_out_params.append(qval(param.item(), 8))

print([qv.dequantize(x) for x in q_in_params])
print([param.item() for param in in_params])
print()
print()
print(qv.dequantize(q_out_params[0]), out_params[0].item())

[-0.07086614173228346, -0.28346456692913385, 0.2204724409448819, 0.1968503937007874, -0.18110236220472442, 0.1889763779527559, 0.13385826771653545, 0.11023622047244094, -0.14173228346456693, -0.2204724409448819]
[-0.07025733590126038, -0.28417205810546875, 0.21682080626487732, 0.19404685497283936, -0.18068020045757294, 0.18948307633399963, 0.13393481075763702, 0.11250212788581848, -0.14335717260837555, -0.2212885469198227]


-0.07086614173228346 -0.0699412077665329


In [10]:
# print(q_in_params)
# print(input_test[0])
q_input = [qval(q.item(), 8) for q in input_test]

print([v.val() for v in q_input])

# result = qv.quantize(0.0, 8)
# for param, x in zip(q_in_params, q_input):
#     result = qv.qAdd(result, param * x)
    
# print(qv.dequantize(result))

[0.10236220472440945, 0.1968503937007874, 0.2992125984251969, 0.4015748031496063, 0.5039370078740157, 0.5984251968503937, 0.7007874015748031, 0.8031496062992126, 0.8976377952755905, 1.0]


In [11]:
def execute_layer(q_in_params, q_out_params, input_test, bits=8):
    q_input = [qval(q.item(), bits) for q in input_test]

    result = qval(0.0, bits)
    for param, x in zip(q_in_params, q_input):
        result = (result + param * x)

    return (result + q_out_params[0])

In [12]:
execute_layer(q_in_params, q_out_params, input_test).val()

-0.13147775132759568

In [13]:
import random
random_values = [random.random() for _ in range(10)]
random_input = torch.tensor(random_values)

original = layer(random_input).item()
quantized = execute_layer(q_in_params, q_out_params, random_input).val()

print(original, quantized)
print(f"Diff: {abs(original - quantized)}")

if abs(original - quantized) > 0.3:
    print([x.item() for x in random_input])
    q_input = [qval(x, 8) for x in random_values]

    print([q.val for q in q_input])

0.15422247350215912 0.15198681560153818
Diff: 0.002235657900620941


In [21]:
for _ in range(10000):
    random_values = [random.random() for _ in range(10)]
    random_input = torch.tensor(random_values)

    original = layer(random_input).item()
    quantized = execute_layer(q_in_params, q_out_params, random_input).val()

    # print(original, quantized)
    # print(f"Diff: {abs(original - quantized)}")
    # print(abs(original - quantized))

    if abs(original - quantized) > 0.3:
        print([x.item() for x in random_input])
        q_input = [qval(x, 8) for x in random_values]

        print([q.val for q in q_input])

In [15]:
from qval.qlayer import qdense

input_params = [[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0] for _ in range(10)]
out_params = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
teste = qdense(input_params, out_params)
teste

Input layers: q8
0 [0.10236220472440945, 0.1968503937007874, 0.2992125984251969, 0.4015748031496063, 0.5039370078740157, 0.5984251968503937, 0.7007874015748031, 0.8031496062992126, 0.8976377952755905, 1.0]
1 [0.10236220472440945, 0.1968503937007874, 0.2992125984251969, 0.4015748031496063, 0.5039370078740157, 0.5984251968503937, 0.7007874015748031, 0.8031496062992126, 0.8976377952755905, 1.0]
2 [0.10236220472440945, 0.1968503937007874, 0.2992125984251969, 0.4015748031496063, 0.5039370078740157, 0.5984251968503937, 0.7007874015748031, 0.8031496062992126, 0.8976377952755905, 1.0]
3 [0.10236220472440945, 0.1968503937007874, 0.2992125984251969, 0.4015748031496063, 0.5039370078740157, 0.5984251968503937, 0.7007874015748031, 0.8031496062992126, 0.8976377952755905, 1.0]
4 [0.10236220472440945, 0.1968503937007874, 0.2992125984251969, 0.4015748031496063, 0.5039370078740157, 0.5984251968503937, 0.7007874015748031, 0.8031496062992126, 0.8976377952755905, 1.0]
5 [0.10236220472440945, 0.196850393700

In [16]:
input_test = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 5.0]

output = teste.process(input_test)

[x.val() for x in output]

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]

In [17]:
a = qval(6.5)
b = qval(2.0)

(a * b).val()

0.9844961240310077

In [18]:
a.val()

1.0

In [19]:
from qval import qval
a = qval(6.5, scale=6.0)
print(a.scale)
print(a.val(), a.scale)

6.5
1.0748031496062993 6.5


In [20]:
a = qval(6.5)
b = qval(2.0)